# Scratchwork

In [1]:
from finite_algebras import make_finite_algebra, Group, Ring, Field, generate_prime_field

import numpy as np

In [2]:
import os
aa_path = os.path.join(os.getenv("PYPROJ"), "abstract_algebra")
alg_dir = os.path.join(aa_path, "Algebras")

## Make Finite Algebra 2

### Rings

In [3]:
rng = make_finite_algebra('Powerset Ring 2',
                          'Ring on powerset of {0, 1}',
                          ['{}', '{0}', '{1}', '{0, 1}'],
                          [[0, 1, 2, 3],
                           [1, 0, 3, 2],
                           [2, 3, 0, 1],
                           [3, 2, 1, 0]],
                          [[0, 0, 0, 0],
                           [0, 1, 0, 1],
                           [0, 0, 2, 2],
                           [0, 1, 2, 3]]
                         )
rng.about()

ValueError: '{}' is not in list

## Fields

$GF(4) = GF(2)[X]/(X^2 + X + 1)$

See Wikipedia: ["Field with four elements"](https://en.wikipedia.org/wiki/Finite_field#Field_with_four_elements)

In [ ]:
elems = ['0', '1', 'a', '1+a']

add_table = [[ '0' ,  '1' ,  'a' , '1+a'],
             [ '1' ,  '0' , '1+a',  'a' ],
             [ 'a' , '1+a',  '0' ,  '1' ],
             ['1+a',  'a' ,  '1' ,  '0' ]]

mult_table = [['0',  '0' ,  '0' ,  '0' ],
              ['0',  '1' ,  'a' , '1+a'],
              ['0',  'a' , '1+a',  '1' ],
              ['0', '1+a',  '1' ,  'a' ]]

In [ ]:
f4 = make_finite_algebra_2('F4',
                           'Field with 4 elements',
                           elems,
                           add_table,
                           mult_table
                          )

f4.about()

### Generate Field of Prime Order

In [ ]:
f7 = generate_prime_field(7)
f7.about()

In [ ]:
f7_add = f7.extract_additive_algebra()
f7_add.about()

In [ ]:
f7_mult = f7.extract_multiplicative_algebra()
f7_mult.about()

In [ ]:
f7_mult_subs = f7_mult.proper_subalgebras()

print([(alg.__class__.__name__, alg.order) for alg in f7_mult_subs])

[alg for alg in f7_mult_subs if isinstance(alg, Group)]

In [ ]:
f7_mult_sub1 = f7_mult_subs[1]
f7_mult_sub1.about()